# Covert Degrees in Advance

In [45]:
import time

import numpy as np

In [46]:
def hms2dec(hours, minutes, seconds):
    return 15*(hours + minutes/60 + seconds/(60*60))


def dms2dec(degrees, arcminutes, arcseconds):
    angle = abs(degrees) + arcminutes/60 + arcseconds/(60*60)
    return angle if degrees > 0 else -angle


def angular_dist(ra1, dec1, ra2, dec2):
    a = np.sin(np.abs(dec1 - dec2)/2)**2
    b = np.cos(dec1)*np.cos(dec2)*np.sin(np.abs(ra1 - ra2)/2)**2
    d = 2*np.arcsin(np.sqrt(a + b))
    return np.degrees(d)


def find_closest(catalogue, ra1, dec1):
    closest = (None, np.inf)
    for i, (ra2, dec2) in enumerate(catalogue):
        distance = angular_dist(ra1, dec1, ra2, dec2)
        if distance < closest[1]:
            closest = (i, distance)
    return closest


def crossmatch(catalogue1, catalogue2, max_dist):
    start = time.perf_counter()
    
    matches = []
    no_matches = []
    
    catalogue1 = np.radians(catalogue1)
    catalogue2 = np.radians(catalogue2)
    
    for i, (ra1, dec1) in enumerate(catalogue1):
        j, closest_dist = find_closest(catalogue2, ra1, dec1)
        if closest_dist > max_dist:
            no_matches.append(i)
        else:
            matches.append((i, j, closest_dist))

    time_taken = time.perf_counter() - start
    return matches, no_matches, time_taken

In [47]:
def main():
    cat1 = np.array([[180, 30], [45, 10], [300, -45]])
    cat2 = np.array([[180, 32], [55, 10], [302, -44]])
    matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
    print('matches:', matches)
    print('unmatched:', no_matches)
    print('time taken:', time_taken)


if __name__ == '__main__':
    main()

matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
unmatched: [1]
time taken: 0.00018625299981067656


# Vectorisation

In [48]:
def hms2dec(hours, minutes, seconds):
    return 15*(hours + minutes/60 + seconds/(60*60))


def dms2dec(degrees, arcminutes, arcseconds):
    angle = abs(degrees) + arcminutes/60 + arcseconds/(60*60)
    return angle if degrees > 0 else -angle


def angular_dist(ra1, dec1, ra2, dec2):
    a = np.sin(np.abs(dec1 - dec2)/2)**2
    b = np.cos(dec1)*np.cos(dec2)*np.sin(np.abs(ra1 - ra2)/2)**2
    return 2*np.arcsin(np.sqrt(a + b))


def find_closest(cat2, ra1, dec1):    
    ra2s = cat2[:, 0]
    dec2s = cat2[:, 1]
    dists = angular_dist(ra1, dec1, ra2s, dec2s)
    closest_id = np.argmin(dists)
    return closest_id, dists[closest_id]


def crossmatch(catalogue1, catalogue2, max_dist):
    start = time.perf_counter()
    
    matches = []
    no_matches = []
    
    max_dist = np.radians(max_dist)
    catalogue1 = np.radians(catalogue1)
    catalogue2 = np.radians(catalogue2)
    
    for i, (ra1, dec1) in enumerate(catalogue1):
        j, closest_dist = find_closest(catalogue2, ra1, dec1)
        if closest_dist > max_dist:
            no_matches.append(i)
        else:
            matches.append((i, j, closest_dist))

    time_taken = time.perf_counter() - start
    return matches, no_matches, time_taken

In [49]:
def main():
    ra1, dec1 = np.radians([180, 30])
    cat2 = [[180, 32], [55, 10], [302, -44]]
    cat2 = np.radians(cat2)
    ra2s, dec2s = cat2[:,0], cat2[:,1]
    dists = angular_dist(ra1, dec1, ra2s, dec2s)
    print(np.degrees(dists))
    
    cat1 = np.array([[180, 30], [45, 10], [300, -45]])
    cat2 = np.array([[180, 32], [55, 10], [302, -44]])
    matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
    print('matches:', matches)
    print('unmatched:', no_matches)
    print('time taken:', time_taken)


if __name__ == '__main__':
    main()

[  2.         113.72587199 132.64478705]
matches: [(0, 0, 0.03490658503988664), (2, 2, 0.03040382589186957)]
unmatched: [1]
time taken: 0.0001228339997396688


# Skip Points

In [ ]:
def hms2dec(hours, minutes, seconds):
    return 15*(hours + minutes/60 + seconds/(60*60))


def dms2dec(degrees, arcminutes, arcseconds):
    angle = abs(degrees) + arcminutes/60 + arcseconds/(60*60)
    return angle if degrees > 0 else -angle


def angular_dist(ra1, dec1, ra2, dec2):
    a = np.sin(np.abs(dec1 - dec2)/2)**2
    b = np.cos(dec1)*np.cos(dec2)*np.sin(np.abs(ra1 - ra2)/2)**2
    return 2*np.arcsin(np.sqrt(a + b))


def find_closest(cat2, ra1, dec1, max_dist):
    # cat2 is expected to be sorted
    closest = (None, np.inf)
    for i, (ra2, dec2) in enumerate(cat2):
        if dec2 > dec1 + max_dist:
            break  # if we are too far away already
        distance = angular_dist(ra1, dec1, ra2, dec2)
        if distance < closest[1]:
            closest = (i, distance)
    return closest


def crossmatch(cat1, cat2, max_dist):
    start = time.perf_counter()
    
    matches = []
    no_matches = []
    
    max_dist = np.radians(max_dist)
    cat1 = np.radians(cat1)
    cat2 = np.radians(cat2)
    
    cat2_sorted_ids = np.argsort(cat2[:,1])
    cat2_sorted = cat2[cat2_sorted_ids]
    
    for i, (ra1, dec1) in enumerate(cat1):
        j, closest_dist = find_closest(cat2_sorted, ra1, dec1, max_dist)
        if closest_dist > max_dist:
            no_matches.append(i)
        else:
            old_j = cat2_sorted_ids[j]
            matches.append((i, old_j, closest_dist))

    time_taken = time.perf_counter() - start
    return matches, no_matches, time_taken

In [ ]:
def main():
    cat1 = np.array([[180, 30], [45, 10], [300, -45]])
    cat2 = np.array([[180, 32], [55, 10], [302, -44]])
    matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
    print('matches:', matches)
    print('unmatched:', no_matches)
    print('time taken:', time_taken)

    def create_cat(n):
        """Create random catalogue."""
        ras = np.random.uniform(0, 360, size=(n, 1))
        decs = np.random.uniform(-90, 90, size=(n, 1))
        return np.hstack((ras, decs))

    # Test on random inputs
    np.random.seed(0)
    cat1 = create_cat(10)
    cat2 = create_cat(20)
    matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
    print('matches:', matches)
    print('unmatched:', no_matches)
    print('time taken:', time_taken)

if __name__ == '__main__':
    main()